<a href="https://colab.research.google.com/github/dlsanf2000/HIR/blob/main/i2b2_ELMo_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd

In [20]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [21]:
# data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bbc-text.csv')
import ast
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sep_df_Train.csv') #Set Dir
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sep_df_Test.csv') #Set Dir

In [22]:
train_df['words'] = train_df['words'].apply(ast.literal_eval)
train_df['tags'] = train_df['tags'].apply(ast.literal_eval)
test_df['words'] = test_df['words'].apply(ast.literal_eval)
test_df['tags'] = test_df['tags'].apply(ast.literal_eval)

train_words = train_df.words
train_tags = train_df.tags

test_words = test_df.words
test_tags = test_df.tags

In [23]:
words = list(train_words) + list(test_words)
tags = list(train_tags) + list(test_tags)

In [24]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [25]:
!pip uninstall tensorflow tensorflow_hub tensorflowjs
!pip install tensorflow==2.0.0a0 tensorflow_hub==0.5.0 tensorflowjs==1.2.6

Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
    /usr/local/lib/python3.7/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0
Uninstalling tensorflow-hub-0.11.0:
  Would remove:
    /usr/local/bin/make_image_classifier
    /usr/local/bin/make_nearest_neighbour_index
    /usr/local/lib/python3.7/dist-packages/tensorflow_hub-0.11.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow_hub/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-hub-0.11.0
Uninstalling tensorflowjs-1.2.6:
  Would remove:
    /usr/local/bin/tensorflowjs_conver

In [26]:
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np


  Using cached https://files.pythonhosted.org/packages/92/2b/e3af15221da9ff323521565fa3324b0d7c7c5b1d7a8ca66984c8d59cb0ce/tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl
ERROR: tensorflowjs 1.2.6 has requirement numpy==1.16.4, but you'll have numpy 1.19.5 which is incompatible.
ERROR: tensorflowjs 1.2.6 has requirement six==1.11.0, but you'll have six 1.15.0 which is incompatible.
ERROR: tensorflowjs 1.2.6 has requirement tensorflow==1.14.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: tensorflow 2.0.0a0
    Uninstalling tensorflow-2.0.0a0:
      Successfully uninstalled tensorflow-2.0.0a0


  Using cached https://files.pythonhosted.org/packages/f4/1f/ca72fa5400954903aa6f3354142cd66374604384fb13429af9e41b12120d/tensorflow_hub-0.11.0-py2.py3-none-any.whl
ERROR: tensorflowjs 1.2.6 has requirement numpy==1.16.4, but you'll have numpy 1.19.5 which is incompatible.
ERROR: tensorflowjs 1.2.6 has requirement six==1.11.0, but you'll have six 1.15.0 which is incompatible.
ERROR: tensorflowjs 1.2.6 has requirement tensorflow==1.14.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflowjs 1.2.6 has requirement tensorflow-hub==0.5.0, but you'll have tensorflow-hub 0.11.0 which is incompatible.
  Found existing installation: tensorflow-hub 0.5.0
    Uninstalling tensorflow-hub-0.5.0:
      Successfully uninstalled tensorflow-hub-0.5.0


In [27]:
!pip install tensorflow==1.15

In [28]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
# print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  1.15.0
Eager mode:  False
Hub version:  0.11.0


In [29]:
import tensorflow_hub as hub
import tensorflow as tf

embed = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
# elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/2", name="ELMO")

In [30]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [31]:
# le = preprocessing.LabelEncoder()
# le.fit(y)

# def encode(le, labels):
#     enc = le.transform(labels)
#     return keras.utils.to_categorical(enc)

# def decode(le, one_hot):
#     dec = np.argmax(one_hot, axis=1)
#     return le.inverse_transform(dec)


# x_enc = x
# y_enc = encode(le, y)

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_words, test_words, train_tags, test_tags

In [33]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), as_dict = True, signature = "default")["default"]

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(2379, activation='relu')(embedding)
pred = Dense(1, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [34]:
# from keras.utils import to_categorical

# with tf.Session() as session:
#     K.set_session(session)
#     session.run(tf.global_variables_initializer())  
#     session.run(tf.tables_initializer())
#     y_train = to_categorical(y_train)
#     history = model.fit(X_train, y_train, epochs=1, batch_size=16)
#     model.save_weights('./elmo-model.h5')

# with tf.Session() as session:
#     K.set_session(session)
#     session.run(tf.global_variables_initializer())
#     session.run(tf.tables_initializer())
#     model.load_weights('./elmo-model.h5')  
#     predicts = model.predict(X_test, batch_size=16)

# y_test = decode(le, y_test)
# y_preds = decode(le, predicts)



ValueError: ignored

In [38]:
y_train.reshape()

(2379,)

In [35]:
history = model.fit(X_train, y_train, epochs = 1, batch_size = 16)

ValueError: ignored

In [ ]:
predicts = model.predict(X_test, batch_size=16)

y_test = decode(le, y_test)
y_preds = decode(le, predicts, axis = 0)

In [ ]:
# from sklearn import metrics

# print(metrics.confusion_matrix(y_test, y_preds))

# print(metrics.classification_report(y_test, y_preds))

# from sklearn.metrics import accuracy_score

# print("Accuracy of ELMO is:",accuracy_score(y_test,y_preds))